In [1]:
%load_ext autoreload
%autoreload 2

In [160]:
import instructor
import google.generativeai as genai
from pydantic import BaseModel
from rich import print

client = instructor.from_gemini(
    genai.GenerativeModel("gemini-2.0-flash-exp"), mode=instructor.Mode.GEMINI_JSON
)


class StoryOutline(BaseModel):
    title: str
    description: str
    banner_image_description: str


user_prompt = (
    "write me a story about a man who finds himself in a post apocalyptic world"
)


resp = client.chat.completions.create(
    response_model=StoryOutline,
    messages=[
        {
            "role": "system",
            "content": """
Here is a prompt provided by a user who wants to play an adventure game.

<prompt>
{{ prompt }}
</prompt>

Read the prompt carefully, identify specific details and elements and then generate the following

- A title for the story that's between 3-6 words
- A description for the story that's between 3-5 sentences. In this description, you must introduce the main character and set the scene. Make sure to mention the main character's name and what's at stake for them here in this existing situation implicitly.
- A short 1 sentence  description for a banner image . This should be a description of a pixel art image that's suitable for the story as cover art. Be specific about the colors, style of the image, individual components of the image and the background.
            """,
        }
    ],
    context={"prompt": user_prompt},
)

print(resp)

StoryOutline(
    title='Wasteland Wanderer: Lost Echoes',
    description='John wakes up disoriented in a desolate, ruined city. The world is ravaged, with skeletal 
buildings piercing a dust-filled sky. He has no memory of how he got here or who he is. Survival in this hostile 
wasteland is his only focus, as he struggles to uncover the secrets of his past, hoping to find what remains.',
    banner_image_description='A pixel art image in muted browns and grays, depicting a lone figure in tattered 
clothing standing on a broken highway, with skeletal buildings silhouetted against a hazy orange sky.'
)

In [188]:
from pydantic import BaseModel, field_validator, ValidationInfo


class UserChoice(BaseModel):
    choice_title: str


class StoryNode(BaseModel):
    title: str
    story_description: str
    banner_image_description: str
    user_choices: list[UserChoice]

    @field_validator("user_choices")
    def validate_user_choices(cls, v, info: ValidationInfo):
        context = info.context
        if len(v) != 2 and context["remaining_turns"] > 0:
            raise ValueError("Only provide two choices to the user")

        if len(v) == 0 and context["remaining_turns"] != 0:
            raise ValueError(
                "You must provide two choices for the user to advance the story"
            )

        return v


async def initialize_story(
    client, story_title: str, story_description: str, remaining_turns: int
) -> StoryNode:
    return await client.chat.completions.create(
        response_model=StoryNode,
        messages=[
            {
                "role": "system",
                "content": """
Here's a story outline that we've generated previously based on a user prompt

<outline>
Title: {{ story_title }}
Description: {{ story_description }}
</outline>

Based on the outline above, generate the following:

- A description and title of a new chapter in the story that picks off where the description below ends. This should be a continuation of the story above and between 4-6 sentences.
- Two choices that the user can make at that point in time to advance the story. These titles should be between 3-6 words.
- A banner image description of about 15 words that's suitable for the story as cover art. This should be in a pixel art and retro 8-bit style. Mention specific details of the image in the description.
            """,
            }
        ],
        context={
            "story_title": story_title,
            "story_description": story_description,
            "remaining_turns": remaining_turns,
        },
    )


client = instructor.from_gemini(
    genai.GenerativeModel("gemini-2.0-flash-exp"), use_async=True
)
choice1 = await initialize_story(client, resp.title, resp.description, 2)
print(choice1)

StoryNode(
    title='The Library Vault',
    story_description="John scavenges through the ruins, each crumbling wall and broken window a reminder of the 
world's end. He finds a tattered map hinting at a hidden vault beneath the old city library. A sense of hope sparks
within him, a beacon in the dimness of his amnesia, as he decides to follow the map's directions. He steps into the
library with the hope that this vault will hold the key to his past. The air grows heavy, the silence is thick, and
a strange symbol is etched on the vault door.",
    banner_image_description='A pixel art image of a desolate cityscape at sunset with a lone figure in the 
foreground, rendered in 8-bit style.',
    user_choices=[UserChoice(choice_title='Examine the Symbol'), UserChoice(choice_title='Force the Vault Door')]
)

In [187]:
async def continue_story(
    client, story_title: str, story_description: str, previous_choices: list[dict]
) -> StoryNode:
    remaining_turns = 3 - len(previous_choices)
    print(f"Generating story node with {remaining_turns} turns remaining")
    return await client.chat.completions.create(
        response_model=StoryNode,
        messages=[
            {
                "role": "system",
                "content": """
Here's a story outline that we've generated previously based on a user prompt

<outline>
Title: {{ story_title }}
Description: {{ story_description }}
</outline>

Based on the outline above, generate the following:

- A description and title of a new chapter in the story that picks off where the description below ends. This should be a continuation of the story above and between 3-5 sentences.
- Two choices that the user can make at that point in time to advance the story. These titles should be between 3-6 words. Make sure to reference specific elements mentioned in the generated description of the story chapter.
- A banner image description of about 15 words that's suitable for the story as cover art. This should be in a pixel art and retro 8-bit style. Mention specific details of the image in the description.

{% if previous_choices | length >= 1 %}
Here are the previous choices made by the main character leading up to this point in the story. Read them carefully and make sure to reference specific elements mentioned in the generated description of the new story chapter.
{% endif %}

<previous choices>
    {% for choice in previous_choices %}
    <choice {{loop.index}}>
    Choice Context: {{ choice.context }}
    Options: {{ choice.options }}
    User Chose: {{ choice.user_choice }}
    </choice>
{% endfor %}
<previous choices>
            """,
            }
        ],
        context={
            "story_title": story_title,
            "story_description": story_description,
            "previous_choices": previous_choices,
            "remaining_turns": remaining_turns,
        },
    )


client = instructor.from_gemini(
    genai.GenerativeModel("gemini-2.0-flash-exp"), use_async=True
)
choice2 = await continue_story(
    client,
    resp.title,
    resp.description,
    [
        {
            "title": choice1.title,
            "description": choice1.story_description,
            "options": [choice.choice_title for choice in choice1.user_choices],
            "user_choice": choice1.user_choices[0].choice_title,
        }
    ],
)
print(choice2)

Generating story node with 2 turns remaining

StoryNode(
    title='Echoes of the Past',
    story_description='Dust hangs heavy in the air as John studies the mural. Its faded colors depict a city much 
like this one, but thriving. The central image is a tower reaching to the sky, with a symbol at its peak that looks
familiar somehow. A low growl echoes from the shadows, and a pair of glowing red eyes pierce the gloom.',
    banner_image_description='A pixel art banner of a crumbling city with a lone figure, John, in the foreground, 
looking at an old mural.',
    user_choices=[UserChoice(choice_title='Approach the Eyes'), UserChoice(choice_title='Examine the Symbol')]
)

In [189]:
choice3 = await continue_story(
    client,
    resp.title,
    resp.description,
    [
        {
            "title": choice1.title,
            "description": choice1.story_description,
            "options": [choice.choice_title for choice in choice1.user_choices],
            "user_choice": choice1.user_choices[0].choice_title,
        },
        {
            "title": choice2.title,
            "description": choice2.story_description,
            "options": [choice.choice_title for choice in choice2.user_choices],
            "user_choice": choice2.user_choices[0].choice_title,
        },
    ],
)
print(choice3)

Generating story node with 1 turns remaining

StoryNode(
    title='Whispers of the Past',
    story_description="John cautiously approaches the glowing eyes, and as he gets closer, a voice echoes in his 
mind. It's a fragmented memory, a woman's voice calling out a name, not his own. The voice speaks of a hidden 
vault, deeper within the city ruins, where the secrets of the past are stored. The eyes fade away, leaving only a 
faint shimmer where they were, and John is left with more questions than answers, and an inkling of hope that his 
true identity might be locked within the vault. He must now decide how to get there, in hopes that he might find 
the truth he seeks.",
    banner_image_description='A pixel art banner image of a desolate cityscape at dusk. Skeletal buildings and a 
hazy sky. A figure with glowing eyes stands in the foreground.',
    user_choices=[
        UserChoice(choice_title='Search for the Vault'),
        UserChoice(choice_title='Follow the Fading Shimmer')
    ]
)

In [190]:

async def end_story(
    client, story_title: str, story_description: str, prior_choices: list[dict]
) -> StoryNode:
    return await client.chat.completions.create(
        response_model=StoryNode,
        messages=[
            {
                "role": "system",
                "content": """
Here's a story outline that we've generated previously based on a user prompt

<outline>
Title: {{ story_title }}
Description: {{ story_description }}
</outline>

Here are the previous choices made by the main character leading up to this point in the story. Read them carefully and make sure to reference specific elements mentioned in your generated description of the story ending.

<previous choices>
    {% for choice in previous_choices %}
    <choice {{loop.index}}>
    Choice Context: {{ choice.context }}
    Choice Options: {{ choice.options }}
    User Chose: {{ choice.user_choice }}
    </choice>
{% endfor %}
<previous choices>

Based on the outline above, generate the following:

- A description of the final chapter of the story that's between 3-5 sentences. This should be a conclusion of the story and tie up all loose ends.
- There should be no choices for the user to make at this point in the story.
- A banner image description of about 15 words that's suitable for the story as cover art. This should be in a pixel art and retro 8-bit style. Mention specific details of the image in the description.
            """,
            }
        ],
        context={
            "story_title": story_title,
            "story_description": story_description,
            "remaining_turns": 0,
            "previous_choices": prior_choices,
        },
    )

end_story = await end_story(client, resp.title, resp.description, [
        {
            "title": choice1.title,
            "description": choice1.story_description,
            "options": [choice.choice_title for choice in choice1.user_choices],
            "user_choice": choice1.user_choices[0].choice_title,
        },
        {
            "title": choice2.title,
            "description": choice2.story_description,
            "options": [choice.choice_title for choice in choice2.user_choices],
            "user_choice": choice2.user_choices[0].choice_title,
        },
        {
            "title": choice3.title,
            "description": choice3.story_description,
            "options": [choice.choice_title for choice in choice3.user_choices],
            "user_choice": choice3.user_choices[0].choice_title,
        },
    ],
)

print(end_story)

StoryNode(
    title="The Vault's Echo",
    story_description="John's journey through the wasteland culminates at the vault, a place hinted at by the 
symbols he encountered. Inside, fragments of his past life return, triggered by a hidden databank. He discovers he 
was part of a project that led to the world's desolation. Accepting his identity, John walks out of the vault, 
carrying the weight of his past, his path ahead still uncertain but filled with purpose.",
    banner_image_description='A lone wanderer stands before a massive vault door, pixelated and metallic, under a 
sky filled with dust clouds.',
    user_choices=[]
)

In [113]:
choices = [
    {
        "options": [
            choice.user_choices[0].choice_title,
            choice.user_choices[1].choice_title,
        ],
        "user_choice": {
            "title": choice.user_choices[0].choice_title,
        },
        "context": choice.description,
    },
    {
        "options": [
            choice2.user_choices[1].choice_title,
            choice2.user_choices[0].choice_title,
        ],
        "user_choice": {
            "title": choice2.user_choices[1].choice_title,
        },
        "context": choice2.description,
    },
]
# client.on("completion:kwargs", lambda *args, **kwargs: print(kwargs))
choice3 = get_story_node(client, resp.title, resp.description, choices, 3)
print(choice3)

Generating final story conclusion

StoryNode(
    title='Sanctuary of the Old World',
    description='Elias, battered but resolute, pressed on through the treacherous landscape, the weight of his 
journal now a source of strength, not just a burden. He navigated through the collapsing structures and the 
remnants of the old world, seeking a place hidden from the reach of those who coveted the past for their own gain. 
Finally, in the heart of the ruined city, he found an old underground bunker, its entrance obscured by debris. 
Inside, he started rebuilding a new sanctuary, not only for the preservation of the knowledge, but also as a beacon
of hope for the future. The stories within his journal would become the foundation of a new world, proving that 
even in the most desolate times, the echoes of the past can light the way forward.',
    user_choices=[]
)